## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np
import sys

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = int(sys.argv[1])

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 45.5 ms ± 41.1 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24999.3936937227

Time taken by reduction operation using numpy.sum(): 91.1 µs ± 889 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24999.39369372282 
 
Time taken by reduction operation using numpy.ndarray.sum(): 72 µs ± 58.9 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24999.39369372282


### EJERCICIO 3.2
#### APARTADO A

In [2]:
from multiprocessing import Pool

if __name__ == "__main__":
    # Caso 1: con un solo proceso
    with Pool(1) as p:
        tiempo1 = %timeit -r 2 -o -q p.map(reduc_operation, [X])
        print("El tiempo utilizando solo un proceso es:", tiempo1)
        
    # Caso 2: con dos procesos
    with Pool(2) as p:
        # Creamos una lista con los subarrays
        subarrays2 = [X[0:int(value/2)], X[int(value/2):value]]
        tiempo2 = %timeit -r 2 -o -q p.map(reduc_operation, subarrays2)
        print("El tiempo utilizando dos procesos es:", tiempo2)
                                          
    # Caso 3: con cuatro procesos
    with Pool(4) as p:
        # Creamos una lista con los subarrays
        subarrays4 = [X[0:int(value/4)], X[int(value/4):int(value/2)], X[int(value/2):int(3*value/4)], X[int(3*value/4):value]]
        tiempo4 = %timeit -r 2 -o -q p.map(reduc_operation, subarrays4)
        print("El tiempo utilizando cuatro procesos es:", tiempo4)

El tiempo utilizando solo un proceso es: 50.6 ms ± 192 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
El tiempo utilizando dos procesos es: 29.6 ms ± 26.6 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
El tiempo utilizando cuatro procesos es: 31.9 ms ± 252 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)


#### APARTADO B

In [3]:
from numba import njit, prange

# Primera versión de la función (con decorador njit)
@njit
def reduc_operation_njit(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

tiempo = %timeit -r 2 -o -q reduc_operation_njit(X)

print("El tiempo utilizando el decorador njit es:", tiempo)

# Segunda versión de la función (con decorador njit y paralelismo)
@njit(parallel=True)
def reduc_operation_njit_p(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

tiempo = %timeit -r 2 -o -q reduc_operation_njit_p(X)

print("El tiempo utilizando el decorador njit y ejecutando en paralelo es:", tiempo)

El tiempo utilizando el decorador njit es: 237 µs ± 32.5 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
El tiempo utilizando el decorador njit y ejecutando en paralelo es: 80.9 ms ± 35.2 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


#### Los resultados de ejecutar el notebook en la cola hpc-bio-pascal mediante sbatch son:
Ejecutando el notebook con 50000000 elementos

Time taken by reduction operation using a function: 45 s ± 17.5 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25000378.309147783

Time taken by reduction operation using numpy.sum(): 59.8 ms ± 56 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 25000378.309139702

Time taken by reduction operation using numpy.ndarray.sum(): 60.4 ms ± 664 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.ndarray.sum(): 25000378.309139702

El tiempo utilizando solo un proceso es: 47.6 s ± 9.02 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

El tiempo utilizando dos procesos es: 25.5 s ± 483 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)

El tiempo utilizando cuatro procesos es: 13.3 s ± 5.74 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

El tiempo utilizando el decorador njit es: 195 ms ± 81.1 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)

El tiempo utilizando el decorador njit y ejecutando en paralelo es: 15.7 ms ± 4.06 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

#### Conclusiones:
Al emplear el paquete multiprocessing se observa una reducción del tiempo conforme se van aumentando el número de procesos. Dicho tiempo se reduce aproximadamente a la mitad cuando el número de procesos se duplica.

Por otro lado, al utilizar el decorador *njit* de *numba* se reduce considerablemente el tiempo, pasando del orden de segundos a milisegundos. Este tiempo se ve aún más reducido cuando se ejecuta en paralelo, suponiendo una reducción de aproximadamente 12X en comparación con su ejecución secuencial con el mismo decorador.